In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from depsurf import CollisionType, InlineType, VersionGroup
from depsurf.output import (
    format_k,
    get_legend_handles_labels,
    label_multiline_text,
    load_df,
    plot_yticks,
    save_fig,
)


def plot_subfig(ax: plt.Axes, df: pd.DataFrame, group: VersionGroup, ylim, lines_fn):
    xs = np.arange(len(df))
    bottom = np.zeros(len(df))
    for col in df.columns:
        ax.bar(xs, df[col], bottom=bottom, label=col, color=col.color)
        bottom += df[col]

    ax.tick_params(axis="both", labelsize=9)
    ax.set_xticks(xs, [group.version_str(v) for v in df.index], rotation=0)
    ax.set_xlabel(group.caption)
    ax.set_xlim(-0.5, len(df) - 0.5)
    ax.set_ylim(0, ylim)

    if lines_fn is None:
        return

    with plt.rc_context({"font.family": "monospace"}):
        for i, (index, row) in enumerate(df.iterrows()):
            lines, colors = zip(*lines_fn(row))
            label_multiline_text(ax, i, ylim, lines, colors=colors, fontsize=9)


def plot_fig(name: str, lines_fn=None, ylim_mult=1.1):
    df = load_df(name)

    fig, axs = plt.subplots(
        1,
        3,
        figsize=(12, 2.25),
        sharey=True,
        width_ratios=VersionGroup.num_versions(),
        gridspec_kw={"wspace": 0.05, "hspace": 0.075},
    )

    ylim = df.sum(axis=1).max() * ylim_mult
    for ax, group in zip(axs, VersionGroup):
        plot_subfig(ax, df.loc[group], group, ylim=ylim, lines_fn=lines_fn)

    plot_yticks(axs[0])
    axs[0].set_ylabel("Number of Functions Compiled")

    handles, labels = get_legend_handles_labels(fig)
    fig.legend(handles, labels, loc="upper center", ncol=4, bbox_to_anchor=(0.5, 1.02))

    save_fig(fig, name)


plot_fig(
    "inline",
    lines_fn=lambda row: [(f"{format_k(row[InlineType.PARTIAL])}", "darkred")],
    ylim_mult=1.1,
)

plot_fig(
    "dup",
    lines_fn=lambda row: [
        (f"{format_k(row[CollisionType.MIXED])}", "darkgreen"),
        (f"{row[CollisionType.STATIC] / 1000:.1f}k", "midnightblue"),
    ],
    ylim_mult=1.2,
)

[       pd.py:50 ] INFO: Loaded df from /Users/szhong/Downloads/bpf-study/output/inline.pkl
[      mpl.py:76 ] INFO: Saved figure to /Users/szhong/Downloads/bpf-study/paper/figs/inline.pdf


ValueError: 'Header Duplication' is not a valid CollisionType